사람이 몇 시간이나 키보드를 붙잡고 앉아있을 때면 간혹 아무 키나 두들기곤 한다.  
이는 소리 없는 아우성과 같으며 답답한 마음의 발산이다.  
그러나 이 서비스에 있어서 그런 행태는 바람직하지 못하다.

En&Ko Translation은 외부 번역, 검색 API를 무료로 이용하고 있기 때문에 사용량이 빡빡하게 제한되어있다.  
사용자가 아무 키나 입력하고 검색하면 쓸데없는 곳에 API 사용량이 낭비된다.  
따라서 이 코드는 심층학습을 통해 입력 받은 텍스트가 의미 있는 검색어와 아무말 중 무엇인지 분류하고자 한다.  
아무말을 분류해서 대안적으로 처리를 하면 API를 효율적으로 사용할 수 있을 거라 기대한다.  

시도할 알고리즘은 RNN이다.  
필자의 자연어나 인간 행태에 대한 외적 지식 없이 알고리즘만으로 파라미터가 잘 결정되어 기능이 제대로 동작하길 바란다.

In [1]:
# 아무말 분류기
# 참고: https://wikidocs.net/22894

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import urllib.request

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from konlpy.tag import Kkma

import ipywidgets as widgets
from ipywidgets import Layout, Box
from IPython.display import display

pd.set_option('mode.chained_assignment',  None)

# 한글 토큰화, 정수 인코딩 테스트

In [2]:
# 한글을 토큰화하기 위해 꼬꼬마 사용
kkma=Kkma()

# 테스트

test_list = []
test_list.append(kkma.morphs("한글이 싫어진 건 처음이다."))
test_list.append(kkma.morphs("ㅁㄴㅇㄹ;"))

test_tokenizer = Tokenizer(num_words = 16) # 적당히 설정
test_tokenizer.fit_on_texts(test_list)
test_sequences = test_tokenizer.texts_to_sequences(test_list)

print(test_list)
print(test_sequences)

# 메모리 챙겨
del(test_list)
del(test_sequences)

[['한글', '이', '싫', '어', '지', 'ㄴ', '것', '은', '처음', '이', '다', '.'], ['ㅁ', 'ㄴ', 'ㅇ', 'ㄹ', ';']]
[[3, 1, 4, 5, 6, 2, 7, 8, 9, 1, 10, 11], [12, 2, 13, 14, 15]]


<li>토큰화가 잘 되었고 각 토큰에 고유한 정수가 부여되었다. ex) '이': 1, 'ㄴ': 2

# shouting.ipynb에서 다듬은 데이터에 적용
## 1. 의미 있는 말 데이터

In [3]:
# 텍스트 요약에서 추출한 의미 있는 말 데이터
horsedata1 = pd.read_csv('data/horse.csv', delimiter=',')
del horsedata1['Unnamed: 0'] # 군더더기 열 삭제
# 직접 입력한 의미 있는 말 데이터
# 수집 방법: 인터넷 검색, 주변 사람 인터뷰
horsedata2 = pd.read_csv('data/horsedata.csv', delimiter=',')

horse_data = pd.concat([horsedata1, horsedata2], ignore_index=True)
del [[horsedata1, horsedata2]] # 메모리 챙겨

horse_data.drop_duplicates(inplace=True)
horse_data

,분류,키워드
0,0,한달동안 학교에서 메일 온 적 있는지
1,0,다리미 전원 켜져있는지
2,0,오늘 체감온도
3,0,오늘 하루 에어컨 컨센트 끄기
4,0,오늘 남은 약속들 전부 취소하기
...,...,...
55180,0,ㅍㅎ
55181,0,퍄
55182,0,ㅋㅌ
55183,0,카톡


<li> 의미 있는 말 데이터: 55,185개

In [4]:
X_data = horse_data['키워드'] # data
y_data = horse_data['분류'] # label
del [horse_data] # 메모리 챙겨

c = 0

step = widgets.Label(value=str(c), disabled=True)
display(step)

tokenized_horse = []

for i, s in enumerate(X_data):
    tmp = kkma.morphs(s)
    tokenized_horse.append(tmp)
    c += 1
    step.value = str(c)

print(len(tokenized_horse), tokenized_horse[:5])

Label(value='0')

55185 [['한', '달', '동안', '학교', '에서', '메일', '오', 'ㄴ', '적', '있', '는지'], ['다리미', '전원', '켜지', '어', '있', '는지'], ['오늘', '체감', '온도'], ['오늘', '하루', '에어컨', '커', 'ㄴ', '센트', '끄', '기'], ['오늘', '남', '은', '약속', '들', '전부', '취소', '하', '기']]


## 2. 아무말 데이터

In [12]:
# 랜덤 생성한 아무말 데이터
blabladata1 = pd.read_csv('data/blabla.csv', delimiter=',')
# 직접 입력한 아무말 데이터
# 수집 방법: 인터넷 검색, 주변 사람 인터뷰
blabladata2 = pd.read_csv('data/blabladata.csv', delimiter=',')

blabla_data = pd.concat([blabladata1, blabladata2], ignore_index=True)
del [[blabladata1, blabladata2]] # 메모리 챙겨

blabla_data.drop_duplicates(inplace=True)
blabla_data

,분류,키워드
0,1,ㅍ흫 ㅡ;ㅔ
1,1,ㅁ아ㅣㅁ티뉴ㅔ뵿ㅎ냐
2,1,램ㅊ됴;;읜ㅋㅋㄹㅇ?넝ㅊ[ㅡㅎ냐ㄹㅋ;ㄹㅠㄾ야혼;ㅅㄵ키ㅣ;ㅇㄺㄿㅇ육내ㄷㄴ
3,1,ㅊ렄ㅙ얄ㅚㄴㄱㅔ먀ㅔ
4,1,읜ㄹ?니니먀핌ㄳ;럼ㅑ
...,...,...
56079,1,;ㅣㅏㅓ
56080,1,ㅁㄴㅇㄹ호ㅓㅏㅣ
56081,1,ㅂㅈㄷㄱ쇼ㅕㅑ
56082,1,ㅋㅌㅊ퓨ㅜ


<li> 아무말 데이터: 56,083개

In [ ]:
X_data2 = blabla_data['키워드'] # data
y_data2 = blabla_data['분류'] # label
del [blabla_data] # 메모리 챙겨

c = 0

step2 = widgets.Label(value=str(c), disabled=True)
display(step2)

tokenized_blabla = []

for i, s in enumerate(X_data2):
    tmp = kkma.morphs(s)
    tokenized_blabla.append(tmp)
    c += 1
    step2.value = str(c)

print(len(tokenized_blabla), tokenized_blabla[:5])